# Process the datasets for masking 

In [1]:
from datasets import concatenate_datasets, load_dataset, Dataset
import os
import re
import json 
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
import torch 
import numpy as np
import evaluate

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
def clean_sentence(sentence):
    suffix = r'(\$_\S*)'
    sentence = re.sub(suffix, '', sentence)
    sentence = sentence.replace("$$", "")
    #sentence = sentence.replace("[", "")
    #sentence = sentence.replace("]", "")
    sentence = sentence.replace("  ", " ")
    suffix2 = r'_[^\s]*'
    sentence = re.sub(suffix2, '', sentence)
    # remove spaces before punctuation
    pattern = r'\s+([.,;?!:])'
    sentence = re.sub(pattern, r'\1', sentence)
    # remove weird ``
    sentence = re.sub(r'``', '"', sentence)
    sentence = re.sub(r"''", '"', sentence)
    sentence = sentence.replace("\/", "")
    return sentence

def process_jsonl(input_file, output_file, col, gol):
    with open(input_file, 'r', encoding='utf-8') as infile, \
         open(output_file, 'a', encoding='utf-8') as outfile:
        
        for line in infile:
            data = json.loads(line)
            data[col] = clean_sentence(data[col])
            data[gol] = clean_sentence(data[gol])
            outfile.write(json.dumps(data) + '\n')

def add_other_golds(input_file, output_file, sentcol, goldcol, finalgoldcol): # only tiger + tüba
    with open(input_file, 'r', encoding='utf-8') as infile, \
         open(output_file, 'a', encoding='utf-8') as outfile:
        for line in infile:
            data = json.loads(line)
            if data[goldcol] != " ":
                json.dump({sentcol: data[sentcol], finalgoldcol: data[goldcol], "FCR": data["FCR"], "Gapping": data["Gapping"], "BCR": data["BCR"], "SGF": data["SGF"]}, outfile)
                outfile.write("\n")

In [15]:
# DELETE FILES BEFORE RUNNING THIS AGAIN!

# print("Getting other gold standards!")
# print("Tiger Train")
# input_file = '/Users/marisa/data/tiger_train.jsonl'
# output_file = '/Users/marisa/data/ALL_tiger_train.jsonl'

# add_other_golds(input_file, output_file, "Original sentence", "gold2 (LCO)", "Canonical form")
# add_other_golds(input_file, output_file, "Original sentence", "Canonical form", "Canonical form")

# print("Tiger Test")
# input_file = '/Users/marisa/data/tiger_test.jsonl'
# output_file = '/Users/marisa/data/ALL_tiger_test.jsonl'

# add_other_golds(input_file, output_file, "Original sentence", "gold2 (LCO)", "Canonical form")
# add_other_golds(input_file, output_file, "Original sentence", "Canonical form", "Canonical form")

# print("TüBa Train")
# input_file = '/Users/marisa/data/tüba_train.jsonl'
# output_file = '/Users/marisa/data/ALL_tüba_train.jsonl'

# add_other_golds(input_file, output_file, "Treebank-Sentence", "Gold_1", "Reconstructed-Sentence")
# add_other_golds(input_file, output_file, "Treebank-Sentence", "Gold_2", "Reconstructed-Sentence")
# add_other_golds(input_file, output_file, "Treebank-Sentence", "Gold_3", "Reconstructed-Sentence")
# add_other_golds(input_file, output_file, "Treebank-Sentence", "Reconstructed-Sentence", "Reconstructed-Sentence")
# print("TüBa Test")
# input_file = '/Users/marisa/data/tüba_test.jsonl'
# output_file = '/Users/marisa/data/ALL_tüba_test.jsonl'

# add_other_golds(input_file, output_file, "Treebank-Sentence", "Gold_1", "Reconstructed-Sentence")
# add_other_golds(input_file, output_file, "Treebank-Sentence", "Gold_2", "Reconstructed-Sentence")
# add_other_golds(input_file, output_file, "Treebank-Sentence", "Gold_3", "Reconstructed-Sentence")
# add_other_golds(input_file, output_file, "Treebank-Sentence", "Reconstructed-Sentence", "Reconstructed-Sentence")

In [19]:
from datasets import Dataset
from collections import defaultdict
import random

feature_columns = ['BCR', 'FCR', 'Gapping', 'SGF']
sentence_counts = defaultdict(int)

def add_prefix_to_duplicates(example):
    global sentence_counts

    sentence = example["Original sentence"]
    gold_standard = example["Canonical form"]

    prefixe = ["Er sagt:", "Sie erzählt:", "Folgendes wird berichtet:"]
    sentence_counts[sentence] += 1
    prefix = random.choice(prefixe)

    modified_sentence = prefix + " " + sentence
    modified_gold_standard = prefix + " " + gold_standard

    print(modified_sentence)
    #print(modified_gold_standard)
    #print("\n")

    modified_example = {key: value for key, value in example.items()}
    modified_example["Original sentence"] = modified_sentence
    modified_example["Canonical form"] = modified_gold_standard
    
    return modified_example

def filter_no_cce(example):
    return all((example[feature] == 0) or example[feature] == "0" for feature in feature_columns)

def balance_datasets(dataset_small, dataset_large, feature_columns):
    balanced_data = []
    
    for feature in feature_columns:
        # Anzahl der Sätze mit Feature = 1 im kleineren Datensatz
        small_subset = dataset_small.filter(lambda x: (x[feature] == 1 ) or (x[feature] == "1"))
        count = len(small_subset)
        #print(feature, count)
        
        # Zufällige Auswahl der gleichen Anzahl aus dem größeren Datensatz
        large_subset = dataset_large.filter(lambda x: (x[feature] == 1 ) or (x[feature] == "1"))
        sampled_large_subset = large_subset.shuffle(seed=42).select(range(min(count, len(large_subset))))
        
        # Kombinieren der Subsets
        balanced_data.append(small_subset)
        balanced_data.append(sampled_large_subset)
    
    # Anzahl der Sätze ohne CCE im kleineren Datensatz
    small_subset = dataset_small.filter(filter_no_cce)
    count = len(small_subset)
    print("NO CCE", count)
    
    # Zufällige Auswahl der gleichen Anzahl aus dem größeren Datensatz
    large_subset = dataset_large.filter(lambda x: (x[feature] == 0 ) or (x[feature] == "0"))
    sampled_large_subset = large_subset.shuffle(seed=42).select(range(min(count, len(large_subset))))
    
    # Kombinieren der Subsets
    balanced_data.append(small_subset)
    balanced_data.append(sampled_large_subset)
    
    # Alle balancierten Subsets zusammenfügen
    combined_dict = {
        key: sum((subset[key] for subset in balanced_data), []) for key in dataset_small.features
    }
    return Dataset.from_dict(combined_dict)

In [4]:
train_data1 = os.path.expanduser("~/data/ALL_tiger_train.jsonl")
train_data2 = os.path.expanduser("~/data/ALL_tüba_train.jsonl")

train_dataset1 = load_dataset("json", data_files=train_data1, split='train')
train_dataset2 = load_dataset("json", data_files=train_data2, split='train')
train_dataset2 = train_dataset2.rename_column("Treebank-Sentence", "Original sentence")
train_dataset2 = train_dataset2.rename_column("Reconstructed-Sentence", "Canonical form")

# Balancierter Datensatz
feature_columns = ['BCR', 'FCR', 'Gapping', 'SGF']
train_dataset = balance_datasets(train_dataset1, train_dataset2, feature_columns) # groß und klein tauschen für erweiterung
train_dataset.map(add_prefix_to_duplicates)
print(train_dataset)
print("Got train data")

train_dataset = train_dataset.shuffle(seed=3)

print("Got train data")

t = "Original sentence"
g = "Canonical form"

test_data1 = os.path.expanduser("~/data/ALL_tiger_test.jsonl")
test_data2 = os.path.expanduser("~/data/ALL_tüba_test.jsonl")
test_dataset1 = load_dataset("json", data_files=test_data1, split='train')
test_dataset2 = load_dataset("json", data_files=test_data2, split='train')
test_dataset2 = test_dataset2.rename_column("Treebank-Sentence", "Original sentence")
test_dataset2 = test_dataset2.rename_column("Reconstructed-Sentence", "Canonical form")        
test_dataset = concatenate_datasets([test_dataset1, test_dataset2])
print("Got test data")

NO CCE 415


Map:   0%|          | 0/6553 [00:00<?, ? examples/s]

Sie erzählt: Wo soviel an- , um- und neugebaut wird , geht das Alte nicht verloren . 
Folgendes wird berichtet: Wenn jedoch , wie es neoliberale Ideologie und Politik mit fundamentalistischer Erbarmungslosigkeit propagiert und durchzusetzen versucht , Produktivitätszuwächse nur die Gewinne überproportional steigen lassen und obendrein Umverteilungen zu Lasten von Einkommen aus abhängiger Beschäftigung und von Einnahmen der öffentlichen Hand und zu Gunsten von Einkommen aus Gewinnen und Vermögen zu verzeichnen sind , muß die Marktwirtschaft aus dem Ruder laufen . 
Er sagt: in allen Bereichen des menschlichen Zusammenlebens sollten wir mit allen verfügbaren Mitteln den Sinn für das bürgerliche Prinzip und für bürgerliche Verantwortung fördern und das Vertrauen des Staates in seine Bürger stärken , ausgehend von einem Glauben an die guten Eigenschaften des Menschen . 
Sie erzählt: Die Bundesländer sollen den Ladenschluß am Samstag um bis zu zwei Stunden vorverlegen oder hinausschieben , a

Map:  69%|██████▉   | 4507/6553 [00:00<00:00, 22614.18 examples/s]

Sie erzählt: Und Nonos Unergründlichkeit assoziiert nicht von ungefähr Numinoses , neigt sich einer sakralisierenden , Zeiten und Räume vergessen machenden , hinter sich lassenden Wahrnehmung - paradoxer Effekt einer aus serieller Rationalisierung hervorgegangenen Tonsprache . 
Sie erzählt: Töpfer , der vor fünf Jahren den Vorsitz der saarländischen CDU übernommen hatte und zweimal als Ministerpräsidentenkandidat vergeblich gegen Lafontaine angetreten war , erinnerte als scheidender Parteivorsitzender an die Schwierigkeiten der Union im seit 1985 SPD-dominierten Saarland . 
Sie erzählt: `` Die Haftungsfrage und der Gläubigerschutz müssen in der Fußball-Bundesliga anders organisiert werden '' , fordert BWL-Professor Scherer , der in Augsburg schon so manchen Studenten bei Diplomarbeiten über dieses Thema begleitet hat und es deshalb ganz genau weiß . 
Er sagt: Wäschefirma will altes Image ablegen und baut Stellen ab 
Sie erzählt: Konsolidierung ist schon zwingend , wenn wir den Staat ha

Map: 100%|██████████| 6553/6553 [00:00<00:00, 22127.46 examples/s]


Dataset({
    features: ['Original sentence', 'Canonical form', 'FCR', 'Gapping', 'BCR', 'SGF'],
    num_rows: 6553
})
Got train data
Got train data
Got test data


In [20]:
print(train_dataset)
print(test_dataset)

Dataset({
    features: ['Original sentence', 'Canonical form', 'FCR', 'Gapping', 'BCR', 'SGF'],
    num_rows: 6553
})
Dataset({
    features: ['Original sentence', 'Canonical form', 'FCR', 'Gapping', 'BCR', 'SGF'],
    num_rows: 1739
})


In [21]:
def insert_extra_id(incomplete, complete):
    incomplete_tokens = incomplete.split()
    complete_tokens = complete.split()
    
    result = []
    extra_id_counter = 0
    i = 0  # Pointer for incomplete_tokens
    j = 0  # Pointer for complete_tokens
    while j < len(complete_tokens):
        if i < len(incomplete_tokens) and incomplete_tokens[i] == complete_tokens[j]:
            result.append(complete_tokens[j])
            i += 1
        else:
            # Collect missing span until tokens match again
            while j < len(complete_tokens) and (i >= len(incomplete_tokens) or complete_tokens[j] != incomplete_tokens[i]):
                j += 1
            result.append(f'<extra_id_{extra_id_counter}>')
            extra_id_counter += 1
            continue
        j += 1
    
    return " ".join(result)

def replace_underscore(match):
    # Extrahiere den Wortstamm und das Suffix
    word, suffix = match.group(1), match.group(2)
    return f"[{word} {suffix}]"

In [22]:
# Verarbeitung des gesamten Datensatzes
processed_dataset = []
for data in train_dataset:
    incomplete = data['Original sentence']
    complete = data['Canonical form']
    processed_input = insert_extra_id(incomplete, complete)

    # add CCE info to the gold sentence
    complete = re.sub(r'(\w+)_(\w+)', replace_underscore, complete)
    
    processed_dataset.append({
        "Masked": processed_input,
        "Target": complete,
        "FCR": data["FCR"],
        "Gapping": data["Gapping"],
        "BCR": data["BCR"],
        "SGF": data["SGF"]
    })

# for entry in processed_dataset:
#     print("Input with extra_id:", entry['Masked'])
#     print("Target:", entry['Target'])
#     print("---")

masked_dataset = Dataset.from_list(processed_dataset)
print(masked_dataset[1000])

processed_test_dataset = []
for data in test_dataset:
    incomplete = data['Original sentence']
    complete = data['Canonical form']
    processed_input = insert_extra_id(incomplete, complete)

    # add cce info to gold sentence 
    complete = re.sub(r'(\w+)_(\w+)', replace_underscore, complete)
    
    processed_test_dataset.append({
        "Masked": processed_input,
        "Target": complete,
        "FCR": data["FCR"],
        "Gapping": data["Gapping"],
        "BCR": data["BCR"],
        "SGF": data["SGF"]
    })

masked_test_dataset = Dataset.from_list(processed_test_dataset)
print(masked_test_dataset[1000])

{'Masked': '- Neben der Herausbildung eines qualifizierten wissenschaftlichen Nachwuchses sollten die bestehende Chancenungleichheit im Bildungswesen abgebaut <extra_id_0> und <extra_id_1> vorhandene Bildungsreserven ausgeschöpft werden .', 'Target': ' - Neben der Herausbildung eines qualifizierten wissenschaftlichen Nachwuchses sollten die bestehende Chancenungleichheit im Bildungswesen abgebaut [werden b] und [neben fg] [der fg] [Herausbildung fg] [eines fg] [qualifizierten fg] [wissenschaftlichen fg] [Nachwuchses fg] [sollten fg]  vorhandene Bildungsreserven ausgeschöpft werden . ', 'FCR': '0', 'Gapping': '0', 'BCR': '1', 'SGF': '0'}
{'Masked': 'Die Durchschnitts-Komödien , -Thriller und -Love Stories - warum sollte jemand dafür seine Wohnung verlassen und <extra_id_0> Geld ausgeben , wenn er sie frei Haus haben konnte ?', 'Target': 'Die Durchschnitts-Komödien , -Thriller und -Love Stories - warum sollte jemand dafür seine Wohnung verlassen und [warum fg] [sollte fg] [jemand fg] [da

In [23]:
# CLEAN SENTENCES
final_dataset = masked_dataset.map(lambda x: {'Masked': clean_sentence(x['Masked'])})
final_dataset = masked_dataset.map(lambda x: {'Target': clean_sentence(x['Target'])})

print(final_dataset[1000])

final_test_dataset = masked_test_dataset.map(lambda x: {'Masked': clean_sentence(x['Masked'])})
final_test_dataset = masked_test_dataset.map(lambda x: {'Target': clean_sentence(x['Target'])})

print(final_test_dataset[1000])

Map: 100%|██████████| 6553/6553 [00:00<00:00, 24083.59 examples/s]


{'Masked': '- Neben der Herausbildung eines qualifizierten wissenschaftlichen Nachwuchses sollten die bestehende Chancenungleichheit im Bildungswesen abgebaut <extra_id_0> und <extra_id_1> vorhandene Bildungsreserven ausgeschöpft werden .', 'Target': ' - Neben der Herausbildung eines qualifizierten wissenschaftlichen Nachwuchses sollten die bestehende Chancenungleichheit im Bildungswesen abgebaut [werden b] und [neben fg] [der fg] [Herausbildung fg] [eines fg] [qualifizierten fg] [wissenschaftlichen fg] [Nachwuchses fg] [sollten fg] vorhandene Bildungsreserven ausgeschöpft werden. ', 'FCR': '0', 'Gapping': '0', 'BCR': '1', 'SGF': '0'}


Map: 100%|██████████| 1739/1739 [00:00<00:00, 23298.86 examples/s]

{'Masked': 'Die Durchschnitts-Komödien , -Thriller und -Love Stories - warum sollte jemand dafür seine Wohnung verlassen und <extra_id_0> Geld ausgeben , wenn er sie frei Haus haben konnte ?', 'Target': 'Die Durchschnitts-Komödien, -Thriller und -Love Stories - warum sollte jemand dafür seine Wohnung verlassen und [warum fg] [sollte fg] [jemand fg] [dafür fg] Geld ausgeben, wenn er sie frei Haus haben konnte? ', 'FCR': '0', 'Gapping': '0', 'BCR': '0', 'SGF': '0'}


In [24]:
final_dataset = final_dataset.train_test_split(test_size=0.2)

final_dataset.save_to_disk("FairMasked+CCE_TrainTestDataset")
final_test_dataset.save_to_disk("Masked+CCE_EvalDataset")

Saving the dataset (1/1 shards): 100%|██████████| 1739/1739 [00:00<00:00, 722239.30 examples/s]


In [25]:
print(final_dataset["train"][0])

{'Masked': 'In den vergangenen fünf Jahren warf er den Regierungen Knüppel zwischen die Beine , <extra_id_0> suchte immer wieder nach Vorwänden , um das Parlament aufzulösen und <extra_id_1> hielt enge Kontakte zu den Generalen .', 'Target': 'In den vergangenen fünf Jahren warf er den Regierungen Knüppel zwischen die Beine, [er s] suchte immer wieder nach Vorwänden, um das Parlament aufzulösen und [er f] hielt enge Kontakte zu den Generalen. ', 'FCR': '1', 'Gapping': '0', 'BCR': '0', 'SGF': '1'}


# First training 

In [23]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["bleu"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

# Tokenizer laden
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Tokenisierung
def tokenize_function(example):
    inputs = tokenizer(example["Masked"], padding="max_length", truncation=True, max_length=128)
    targets = tokenizer(example["Target"], padding="max_length", truncation=True, max_length=128)
    
    inputs["labels"] = targets["input_ids"]
    return inputs

# Tokenisiertes Dataset erstellen
tokenized_dataset = final_dataset.map(tokenize_function, batched=True)
print(tokenized_dataset)

device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

model = T5ForConditionalGeneration.from_pretrained("t5-small").to(device)
log_dir = os.path.expanduser("~/models/" + "FirstMasking" + "/logs")
metric = evaluate.load("bleu")

training_args = TrainingArguments(
    output_dir="FirstMasking",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    save_total_limit=0,
    save_strategy="epoch",
    logging_dir=log_dir,
    logging_strategy="steps",
    logging_steps=100,
    load_best_model_at_end=True,
    push_to_hub=False
)

# Trainer einrichten
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],  # Kann auch ein separater Validierungsdatensatz sein
    tokenizer=tokenizer,
    #compute_metrics=compute_metrics,
)

# Training starten
#trainer.train()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Map: 100%|██████████| 1384/1384 [00:00<00:00, 4562.63 examples/s]


DatasetDict({
    train: Dataset({
        features: ['Masked', 'Target', 'FCR', 'Gapping', 'BCR', 'SGF', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 5536
    })
    test: Dataset({
        features: ['Masked', 'Target', 'FCR', 'Gapping', 'BCR', 'SGF', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1384
    })
})


  1%|▏         | 100/6920 [00:27<30:57,  3.67it/s]

                                                  
  1%|▏         | 100/6920 [00:27<30:57,  3.67it/s]

{'loss': 3.0225, 'learning_rate': 4.9277456647398845e-05, 'epoch': 0.14}


  3%|▎         | 200/6920 [00:54<30:33,  3.66it/s]

                                                  
  3%|▎         | 200/6920 [00:54<30:33,  3.66it/s]

{'loss': 0.3835, 'learning_rate': 4.855491329479769e-05, 'epoch': 0.29}


  4%|▍         | 300/6920 [01:22<30:10,  3.66it/s]

                                                  
  4%|▍         | 300/6920 [01:22<30:10,  3.66it/s]

{'loss': 0.2303, 'learning_rate': 4.783236994219654e-05, 'epoch': 0.43}


  4%|▍         | 301/6920 [01:22<30:17,  3.64it/s]

KeyboardInterrupt: 

In [ ]:
def predict(model, tokenizer, input_text):
    inputs = tokenizer(input_text, return_tensors="pt", max_length=128, truncation=True).to(device)
    outputs = model.generate(**inputs, num_beams=5, num_return_sequences=5)
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

test_input = "Meine Schwester fährt ein rotes Auto und <extra_id_0> wohnt in Bayern."
preds = predict(model, tokenizer, test_input)
for pred in preds:
    print(pred)